<a href="https://colab.research.google.com/github/japnit01/Face-detection-and-recognition/blob/main/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from keras.models import load_model
# from inception_model import model
import matplotlib.pyplot as plt
import os
import numpy as np
import random
from keras.layers import Flatten,Dense,Input,concatenate,Conv2D
from keras.models import Model
from keras.activations import sigmoid

In [3]:
# from sklearn.decomposition import PCA

# class PCAPlotter(tf.keras.callbacks.Callback):
    
#     def __init__(self, plt, embedding_model, x_test, y_test):
#         super(PCAPlotter, self).__init__()
#         self.embedding_model = embedding_model
#         self.x_test = x_test
#         self.y_test = y_test
#         self.fig = plt.figure(figsize=(9, 4))
#         self.ax1 = plt.subplot(1, 2, 1)
#         self.ax2 = plt.subplot(1, 2, 2)
#         plt.ion()
        
#         self.losses = []
    
#     def plot(self, epoch=None, plot_loss=False):
#         x_test_embeddings = self.embedding_model.predict(self.x_test)
#         pca_out = PCA(n_components=2).fit_transform(x_test_embeddings)
#         self.ax1.clear()
#         self.ax1.scatter(pca_out[:, 0], pca_out[:, 1], c=self.y_test, cmap='seismic')
#         if plot_loss:
#             self.ax2.clear()
#             self.ax2.plot(range(epoch), self.losses)
#             self.ax2.set_xlabel('Epochs')
#             self.ax2.set_ylabel('Loss')
#         self.fig.canvas.draw()
    
#     def on_train_begin(self, logs=None):
#         self.losses = []
#         self.fig.show()
#         self.fig.canvas.draw()
#         self.plot()
        
#     def on_epoch_end(self, epoch, logs=None):
#         self.losses.append(logs.get('loss'))
#         self.plot(epoch+1, plot_loss=True)

In [ ]:
img_path = '/home/jap01/Desktop/i/lfw-deepfunneled/lfw-deepfunneled'

x = []
y = []

for folder in os.listdir(img_path):
    print("Loading",folder,"...")
    im_path = img_path +'/'+ folder
    for imgs in os.listdir(im_path):
        image = cv2.imread(im_path + '/' + imgs)
        y.append(folder)
        x.append(image)
x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)

In [4]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# print(x_train.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [5]:
x_train = x_train/255
x_test = x_test/255

In [6]:
x_train = np.expand_dims(x_train,axis=3)
x_test = np.expand_dims(x_train,axis=3)
print(x_train.shape)

(60000, 28, 28, 1)


In [7]:
def create_batch(batch_size=256):
	x_anchors = np.zeros((batch_size,28,28,1))
	x_positives = np.zeros((batch_size,28,28,1))
	x_negatives = np.zeros((batch_size,28,28,1))

	for i in range(0, batch_size):
        
		random_index = random.randint(0, x_train.shape[0] - 1)
		x_anchor = x_train[random_index]
		y = y_train[random_index]

		indices_for_pos = np.squeeze(np.where(y_train == y))
		indices_for_neg = np.squeeze(np.where(y_train != y))

		x_positive = x_train[indices_for_pos[random.randint(0, len(indices_for_pos) - 1)]]
		x_negative = x_train[indices_for_neg[random.randint(0, len(indices_for_neg) - 1)]]

		x_anchors[i] = x_anchor
		x_positives[i] = x_positive
		x_negatives[i] = x_negative

	return [x_anchors, x_positives, x_negatives]


In [8]:
# emb_size = 64

# m =tf.keras.applications.InceptionV3(
#         include_top=False, weights='imagenet', input_tensor=None,
#         input_shape=(96,96,3), pooling=None, classes=10,
#         classifier_activation='softmax'
#     )

# m.summary()

In [9]:
emb_size = 64

m = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(96,96,3),
    pooling=None,
    classes=10,
)

m.summary()

29089792/29084464 [==============================] - 0s 0us/step
Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 48, 48, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 48, 48, 64)   256         conv1/conv[0][0]                 
_______________________

In [10]:
i = Input(shape=(28,28,1))
r = tf.keras.layers.experimental.preprocessing.Resizing(96,96)(i)
print(r.shape)
conv2d = Conv2D(3,(1,1),activation='relu')(r)
m = m(conv2d)

# flatten = m
flatten = Flatten()(m)
embmodel = Dense(64, activation="relu")(flatten)
embmodel = Dense(emb_size, activation='sigmoid')(embmodel)

embmodel = Model(inputs=i, outputs=embmodel)
embmodel.summary()

(None, 96, 96, 1)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
resizing (Resizing)          (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 3)         6         
_________________________________________________________________
densenet121 (Functional)     (None, 3, 3, 1024)        7037504   
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                589888    
_________________________________________________________________
dense_1 (Dense)              (None, 64)    

In [11]:
input_anchor = Input(shape=(28,28,1))
input_positive = Input(shape=(28,28,1))
input_negative = Input(shape=(28,28,1))

embedding_anchor = embmodel(input_anchor)
embedding_positive = embmodel(input_positive)
embedding_negative = embmodel(input_negative)

output = concatenate([embedding_anchor, embedding_positive, embedding_negative], axis=1)

net = Model([input_anchor, input_positive, input_negative], output)
net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 64)           7631558     input_3[0][0]                    
                                                                 input_4[0][0]              

In [12]:
alpha = 0.2

def triplet_loss(y_true, y_pred):
    anchor, positive, negative = y_pred[:,:emb_size], y_pred[:,emb_size:2*emb_size], y_pred[:,2*emb_size:]
    positive_dist = tf.reduce_mean(tf.square(anchor - positive), axis=1)
    negative_dist = tf.reduce_mean(tf.square(anchor - negative), axis=1)
    return tf.maximum(positive_dist - negative_dist + alpha, 0.)

In [13]:
def data_generator(batch_size=32):
    while True:
        x = create_batch(batch_size)
        y = np.zeros((batch_size, 3*emb_size))
        yield x, y

In [14]:
net.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

data_path = r'/content/drive/MyDrive/Colab Notebooks/data'
save_path = os.path.join(data_path, "model_weights_triplet_loss_2048.h5")

In [ ]:
batch_size = 32
epochs = 50
steps_per_epoch = 100
model_history = net.fit(
  data_generator(batch_size),
  epochs=epochs, steps_per_epoch=steps_per_epoch,
  verbose=1
)

Epoch 1/50
100/100 [==============================] - 1310s 13s/step - loss: 0.0600 - accuracy: 0.0023
Epoch 2/50
100/100 [==============================] - 1287s 13s/step - loss: 0.0229 - accuracy: 0.0770
Epoch 3/50
100/100 [==============================] - 1287s 13s/step - loss: 0.0117 - accuracy: 0.1868
Epoch 4/50
100/100 [==============================] - 1282s 13s/step - loss: 0.0104 - accuracy: 0.1753
Epoch 5/50
100/100 [==============================] - 1294s 13s/step - loss: 0.0083 - accuracy: 0.2318
Epoch 6/50
100/100 [==============================] - 1270s 13s/step - loss: 0.0068 - accuracy: 0.3224
Epoch 7/50
100/100 [==============================] - 1271s 13s/step - loss: 0.0086 - accuracy: 0.1815
Epoch 8/50
100/100 [==============================] - 1264s 13s/step - loss: 0.0059 - accuracy: 0.0650
Epoch 9/50
100/100 [==============================] - 1248s 12s/step - loss: 0.0060 - accuracy: 0.1191
Epoch 10/50
100/100 [==============================] - 1251s 13s/step - l

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
net.save_weights(save_path)